In [19]:
from pathlib import Path

import pandas as pd

In [52]:
data_dir = Path.cwd().parent / "runs" / "detect"
exp_name = "yolov5l_freeze_11classes_local_distractor_large_shear_test_data_aug"

In [53]:
class_names = [
    "apple", "banana", "orange", "wheelchair", "wok", "box", "table",
    "tissue", "gas_cylinder", "burner", "cart"
]
class_idx = {name: i for i, name in enumerate(class_names)}

results = []
for path in (data_dir / exp_name / "labels").glob("*.txt"):
    class_name = " ".join(path.stem.split("_")[:2])
    with open(path, "r") as infile:
        predictions = [
            int(x.split()[0]) for x in infile.read().strip().splitlines()
        ]
    pred = "None"
    if class_name == "fruit vendor":
        if (set([class_idx["apple"], class_idx["banana"],
                 class_idx["orange"]]).intersection(predictions)
            and class_idx["table"] in predictions
        ):
            pred = class_name
    elif class_name == "roasted chestnut":
        if (
            class_idx["wok"] in predictions
            and (
                class_idx["gas_cylinder"] in predictions
                or class_idx["burner"] in predictions
            )
        ):
            pred = class_name
    elif class_name == "tissue vendor":
        if (
            class_idx["wheelchair"] in predictions
            and class_idx["tissue"] in predictions
        ):
            pred = class_name
    results.append([path.name, class_name, pred])


In [44]:
def compare_prediction(group):
    count = 0
    for i, row in group.iterrows():
        if row["class"] == row["pred"]:
            count += 1
    return count / len(group)

In [54]:
df = pd.DataFrame(results, columns=["filename", "class", "pred"])

In [55]:
df

,filename,class,pred
0,fruit_vendor_1.txt,fruit vendor,None
1,fruit_vendor_2.txt,fruit vendor,None
2,fruit_vendor_3.txt,fruit vendor,None
3,fruit_vendor_4.txt,fruit vendor,None
4,roasted_chestnut_1.txt,roasted chestnut,roasted chestnut
5,roasted_chestnut_2.txt,roasted chestnut,roasted chestnut
6,roasted_chestnut_3.txt,roasted chestnut,None
7,roasted_chestnut_4.txt,roasted chestnut,roasted chestnut
8,roasted_chestnut_large_1.txt,roasted chestnut,roasted chestnut
9,roasted_chestnut_large_2.txt,roasted chestnut,None


In [56]:
summary = df.groupby("class").apply(compare_prediction)

In [57]:
df.to_csv(data_dir / exp_name / "pred.csv")
summary.to_csv(data_dir / exp_name / "summary.csv")